In [1]:
import pandas as pd
import numpy as np

# Function to calculate ATR
def calculate_atr(high, low, close, period):
    tr1 = high - low
    tr2 = np.abs(high - close.shift(1))
    tr3 = np.abs(low - close.shift(1))
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.rolling(window=period).mean()
    return atr

# Function to calculate AVSO
def calculate_avso(src, length, smoothing_length):
    mean_src = src.rolling(window=length).mean()
    std_src = src.rolling(window=length).std()
    z_score = (src - mean_src) / std_src
    avso = z_score.ewm(span=smoothing_length).mean()
    return avso

# UT Bot Strategy with AVSO
def ut_bot_strategy(data, atr_period, key_value, avso_length, avso_smoothing_length):
    # Calculate ATR and Trailing Stop
    data['ATR'] = calculate_atr(data['High'], data['Low'], data['Close'], atr_period)
    data['TrailingStop'] = np.nan
    trailing_stop = [np.nan]

    for i in range(1, len(data)):
        prev_stop = trailing_stop[-1]
        close = data['Close'][i]
        atr = data['ATR'][i] * key_value

        if close > prev_stop if not pd.isna(prev_stop) else True:
            trailing_stop.append(max(prev_stop, close - atr) if not pd.isna(prev_stop) else close - atr)
        else:
            trailing_stop.append(min(prev_stop, close + atr) if not pd.isna(prev_stop) else close + atr)

    data['TrailingStop'] = trailing_stop

    # Calculate AVSO
    data['AVSO'] = calculate_avso(data['Close'], avso_length, avso_smoothing_length)

    # Buy/Sell Signals
    data['Buy'] = (data['Close'] > data['TrailingStop']) & (data['AVSO'] > 0)
    data['Sell'] = (data['Close'] < data['TrailingStop']) & (data['AVSO'] < 0)

    return data



   Close  High  Low  ATR  TrailingStop  AVSO    Buy   Sell
0    100   101   99  NaN           NaN   NaN  False  False
1    102   103  101  NaN           NaN   NaN  False  False
2    101   102  100  NaN           NaN   NaN  False  False
3    103   104  102  NaN           NaN   NaN  False  False
4    104   105  103  NaN           NaN   NaN  False  False
5    102   103  101  NaN           NaN   NaN  False  False
6    100   101   99  NaN           NaN   NaN  False  False
7     99   100   98  NaN           NaN   NaN  False  False
8    101   102  100  NaN           NaN   NaN  False  False
